In [6]:
import sys
sys.path.append('/home/senzankou/Koofr/uni/NSSC1/exercise3/ASC-ODE-team04/build/mechsystem')

from mass_spring import *
import numpy as np
from pythreejs import *

In [7]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)
l = 1.
h = 1.
v = 8.
mA = mss.add (Mass(1, (l,0,0), (0,v,0.5)))
mB = mss.add (Mass(1, (l*np.cos(2*np.pi/3),l*np.sin(2*np.pi/3),0), (-v*np.sin(2*np.pi/3),v*np.cos(2*np.pi/3),0)))
mC = mss.add (Mass(1, (l*np.cos(4*np.pi/3),l*np.sin(4*np.pi/3),0), (-v*np.sin(4*np.pi/3),v*np.cos(4*np.pi/3),0)))

f1 = mss.add (Fix( (0,0,-h)) )
length = np.sqrt((l*np.cos(2*np.pi/3)-l)**2 + (l*np.sin(2*np.pi/3))**2)
height = np.sqrt(l**2+h**2)
mss.add (Spring(height, 30000, (f1, mA)))
mss.add (Spring(height, 30000, (f1, mB)))
mss.add (Spring(height, 30000, (f1, mC)))
mss.add (Spring(length, 30000, (mA, mB)))
mss.add (Spring(length, 30000, (mB, mC)))
mss.add (Spring(length, 30000, (mC, mA)))

5

In [8]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [9]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [ ]:
from time import sleep
for i in range(10000):
    mss.simulate (0.01, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)